# Reconciling end-of-day positions with an external system

In this training module we'll see how to use LUSID to perform the following task:

**<div align="center">As a portfolio manager, I want to load start-of-day positions, intra-day transactions, and end-of-day positions from an external system, and then reconcile LUSID's holdings calculation with that of the external system.</div>**

In [1]:
# Set up LUSID
import os
import pandas as pd
import json
import uuid
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import logging
logging.basicConfig(level=logging.INFO)
from datetime import datetime, timedelta

import lusid as lu
import lusid.api as la
import lusid.models as lm

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10020.0,0.5.3017,"{'relation': 'RequestLogs', 'href': 'http://ja..."


In [2]:
# Create a scope and code to segregate data in this module from other modules
module_scope = "FBNUniversity"
module_code = "Module-3-2"
print(f"'{module_scope}\{module_code}' scope and code created.")

'FBNUniversity\Module-3-2' scope and code created.


## 1. Examining the source files

In [3]:
# Read start-of-day positions into Pandas dataframe
SOD_positions_df = pd.read_csv("data/positions-sod.csv", keep_default_na = False)
display(SOD_positions_df)

,Asset,Class,Figi,Quantity,Price
0,GBP,Cash,,50000,1.00
1,BP,Equity,BBG000C05BD1,5000,2.00
2,Unilever,Equity,BBG000C0M8X7,4000,3.00


In [4]:
# Read intra-day transactions into dataframe
transactions_df = pd.read_csv("data/transactions.csv", keep_default_na = False)
display(transactions_df)

,instrument,figi,txn_id,txn_type,trade_date,units,price,currency
0,BP,BBG000C05BD1,MD32001,Buy,2022-03-07T12:00:00Z,10000,2.00,GBP
1,Unilever,BBG000C0M8X7,MD32002,Sell,2022-03-07T12:10:00Z,3000,3.00,GBP


In [5]:
# Read end-of-day positions into dataframe
EOD_positions_df = pd.read_csv("data/positions-eod.csv", keep_default_na = False)
display(EOD_positions_df)

,Asset,Class,Figi,Quantity,Price
0,GBP,Cash,,38000,1.00
1,BP,Equity,BBG000C05BD1,14900,2.00
2,Unilever,Equity,BBG000C0M8X7,1010,3.00


## 2. Ensuring data is created correctly

### 2.1 Mastering instruments in a custom scope
It's possible the equity instruments in our source files are already mastered in LUSID as part of the demonstration data, but for the avoidance of doubt we'll master them separately in a segregated custom instrument scope (the `GBP` currency instrument is mastered out-of-the-box, in the `default` instrument scope).

In [6]:
# Obtain the LUSID Instruments API
instruments_api = api_factory.build(la.InstrumentsApi)

# Create a dictionary of instrument definitions
definitions = {}

# Iterate over each row in the start-of-day positions dataframe
for index, security in SOD_positions_df.iterrows():

    # Model equities
    if security["Class"] == "Equity":
        # Create definitions
        definitions[security["Asset"]] = lm.InstrumentDefinition(
            name = security["Asset"],
            identifiers = {
                "Figi": lm.InstrumentIdValue(value = security["Figi"]),
            },
            definition = lm.Equity(
                instrument_type = "Equity",
                dom_ccy = "GBP",
                identifiers = {}
            )
        )

# Upsert instruments to LUSID
upsert_instruments_response = instruments_api.upsert_instruments(
    request_body = definitions,
    # Master the instruments in a custom scope
    scope = f"{module_scope}{module_code}",
)

# Transform API response to a dataframe and show internally-generated unique LUID for each mastered instrument
upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instruments_response.values.values()))
display(upsert_instruments_response_df[["name", "lusid_instrument_id"]])

,name,lusid_instrument_id
0,Unilever,LUID_00003D95
1,BP,LUID_00003D94


### 2.2 Creating or updating cut labels

A 'cut label' replaces the time portion of a datetime with a meaningful name, to make working across time zones more intuitive. The cut labels we need might also be present as part of the demonstration data, but we'll upsert them again to make sure.

In [7]:
# Obtain the LUSID CutLabelDefinitions API
cut_label_definition_api = api_factory.build(la.CutLabelDefinitionsApi)

# Create a convenience function to either create a cut label or update an existing one
def create_or_update_cut_label(code, description, name, time, zone):
    # Create new cut label if one with the same code doesn't exist...
    try:
        request = lm.CutLabelDefinition(
            code = code, 
            description = description, 
            display_name = name,
            cut_local_time = lm.CutLocalTime(
                hours = time[0:2],
                minutes = time[3:5]
            ),
            time_zone = zone,
        )
        cut_label_definition_api.create_cut_label_definition(
            create_cut_label_definition_request = request
        )
        print(f"Cut label with code {code} created.")
    # ...else update existing cut label
    except lu.ApiException as e:
        request = lm.UpdateCutLabelDefinitionRequest(
            display_name = name,
            description = description,
            cut_local_time = lm.CutLocalTime(
                hours = time[0:2],
                minutes = time[3:5]
            ),
            time_zone = zone,
        )
        cut_label_definition_api.update_cut_label_definition(
            code = code,
            update_cut_label_definition_request = request
        )
        print(f"Cut label with code {code} updated.")

create_or_update_cut_label("LDN_Open", "LondonOpen", "London Market Open Time", "08:00", "Europe/London")
create_or_update_cut_label("LDN_Close", "LondonClose", "London Market Close Time", "16:30", "Europe/London")

Cut label with code LDN_Open updated.
Cut label with code LDN_Close updated.


## 3. Creating a suitable portfolio
We must set the instrument scope of the portfolio to be the custom scope in which we mastered our instruments. LUSID then attempts to resolve transactions and holdings in the portfolio to instruments in the custom scope.

In [8]:
# Obtain the LUSID Transaction Portfolio API
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)

# Create portfolio definition
portfolio_definition=lm.CreateTransactionPortfolioRequest(
    display_name="Training module 3.2",
    code = module_code,
    base_currency = "GBP",
    # Must be before first transaction recorded
    created="2022-01-01",
    # Attempt to resolve transactions to instruments in the custom scope before falling back to the default scope
    instrument_scopes = [f"{module_scope}{module_code}"],
)

# Upsert portfolio to LUSID, making sure it's not already there
try:
    create_portfolio_response=transaction_portfolios_api.create_portfolio(
        scope = module_scope,
        create_transaction_portfolio_request = portfolio_definition
    )
    # Confirm success
    print(f"Portfolio with display name '{create_portfolio_response.display_name}' created effective {str(create_portfolio_response.created)}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
            logging.info(json.loads(e.body)["title"])

Portfolio with display name 'Training module 3.2' created effective 2022-01-01 00:00:00+00:00


## 4. Loading start-of-day positions
We can call the LUSID `SetHoldings` API with the built-in `LDN_Open` cut label to set the datetime of each adjustment transaction to precisely 8:00am UTC on 7 March 2022.

In [9]:
# Create convenience function to call
def load_positions(dataframe, cutlabel):
    
    holdings = []
    
    # Iterate over rows in the dataframe, creating one adjustment transaction per row
    for index, row in dataframe.iterrows():
        
        # Specify different identifiers for equities and cash
        if row["Class"] == "Cash":
            identifiers = {"Instrument/default/Currency": row["Asset"]}
        else:
            identifiers = {"Instrument/default/Figi": row["Figi"]}

        holdings.append(
            lm.AdjustHoldingRequest(
                instrument_identifiers = identifiers,
                tax_lots = [
                    lm.TargetTaxLotRequest(
                        units = row["Quantity"],
                        cost = lm.CurrencyAndAmount(
                            # Calculate cost on the fly
                            amount = row["Quantity"] * row["Price"],
                            # Have to set holding currency to same as transaction currency, even if cost basis is 0
                            currency = "GBP"
                        ),
                        portfolio_cost = row["Quantity"] * row["Price"],
                        price = row["Price"]
                    )
                ]
            )
        )

    # Set holdings in LUSID (restate any existing holdings)
    set_holdings_response=transaction_portfolios_api.set_holdings(
        scope = module_scope,
        code = module_code,
        # Make holdings effective from the time of the cut label on 7 March 2022. Note use of 'N' separator
        # between date and cut label time
        effective_at = f"2022-03-07N{cutlabel}",
        adjust_holding_request = holdings
    )
    
    # Confirm by calling GetHoldings
    get_holdings_response=transaction_portfolios_api.get_holdings(
        scope = module_scope, 
        code = module_code,
        effective_at = f"2022-03-07N{cutlabel}",
        # Decorate on instrument name property to make more results more intuitive
        property_keys = ["Instrument/default/Name"]
    )
    # Transform GetHoldings response to a Pandas dataframe and show it
    get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
    # Drop some noisy columns
    get_holdings_response_df.drop(columns=[
        "instrument_scope", "sub_holding_keys", "cost_portfolio_ccy.currency", "currency", "SourcePortfolioId(default-Properties)", "SourcePortfolioScope(default-Properties)"], inplace=True)
    display(get_holdings_response_df)

# Load start-of-day positions into LUSID effective 8:00am UTC
load_positions(SOD_positions_df, "LDN_Open")

,instrument_uid,Name(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount
0,LUID_00003D94,BP,P,"5,000.00","5,000.00","10,000.00",GBP,"10,000.00"
1,LUID_00003D95,Unilever,P,"4,000.00","4,000.00","12,000.00",GBP,"12,000.00"
2,CCY_GBP,GBP,B,"50,000.00","50,000.00","50,000.00",GBP,"50,000.00"


## 5. Loading intra-day transactions

In [10]:
# Create convenience function to call
def load_transactions_from_source_file(vendor_dataframe):
    
    # Iterate over rows in the dataframe, creating one transaction per row
    transactions = [
        lm.TransactionRequest(
            transaction_id = row["txn_id"],
            type = row["txn_type"],
            instrument_identifiers = {"Instrument/default/Figi": row["figi"]},
            # Use LPT to_date function to convert to UTC datetime
            transaction_date = to_date(row["trade_date"]),
            # Settlement date is 2 days later
            settlement_date = to_date(row["trade_date"]) + timedelta(days = 2),
            units = row["units"],
            transaction_price = lm.TransactionPrice(price = row["price"], type="Price"),
            total_consideration = lm.CurrencyAndAmount(
                # Calculate cost on the fly
                amount = row["units"] * row["price"],
                currency = row["currency"]
            ),
            transaction_currency = row["currency"]
        )
        for index, row in vendor_dataframe.iterrows()
    ]

    # Upsert transactions to LUSID
    upsert_transactions_response = transaction_portfolios_api.upsert_transactions(
        scope = module_scope, 
        code = module_code, 
        transaction_request = transactions
    )
        
    # Confirm by calling GetHoldings
    get_holdings_response=transaction_portfolios_api.get_holdings(
        scope = module_scope, 
        code = module_code,
        # Note use of 'T' separator between date and explicit time
        effective_at = "2022-03-07T16:29:00Z",
        property_keys = ["Instrument/default/Name"]
    )
    # Transform API response to a pandas dataframe and show it
    get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
    # Drop some noisy columns
    get_holdings_response_df.drop(columns=[
        "instrument_scope", "sub_holding_keys", "cost_portfolio_ccy.currency", "currency", "SourcePortfolioId(default-Properties)", "SourcePortfolioScope(default-Properties)"], inplace=True)
    display(get_holdings_response_df)
    
# Load intra-day transactions   
load_transactions_from_source_file(transactions_df)

,instrument_uid,Name(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Figi,transaction.instrument_scope,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/ResultantHolding.key,transaction.properties.Transaction/default/ResultantHolding.value.metric_value.value,transaction.entry_date_time,transaction.transaction_status
0,LUID_00003D94,BP,P,"15,000.00","5,000.00","30,000.00",GBP,"30,000.00",NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,LUID_00003D95,Unilever,P,"1,000.00","4,000.00","3,000.00",GBP,"3,000.00",NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,CCY_GBP,GBP,B,"50,000.00","50,000.00","50,000.00",GBP,"50,000.00",NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,CCY_GBP,GBP,C,"-20,000.00",0.00,"-20,000.00",GBP,"-20,000.00",MD32001,Buy,BBG000C05BD1,FBNUniversityModule-3-2,LUID_00003D94,2022-03-07 12:00:00+00:00,2022-03-09 12:00:00+00:00,"10,000.00",2.00,Price,"20,000.00",GBP,1.00,GBP,Transaction/default/ResultantHolding,"15,000.00",2022-09-23 09:42:53.895165+00:00,Active
4,CCY_GBP,GBP,C,"9,000.00",0.00,"9,000.00",GBP,"9,000.00",MD32002,Sell,BBG000C0M8X7,FBNUniversityModule-3-2,LUID_00003D95,2022-03-07 12:10:00+00:00,2022-03-09 12:10:00+00:00,"3,000.00",3.00,Price,"9,000.00",GBP,1.00,GBP,Transaction/default/ResultantHolding,"1,000.00",2022-09-23 09:42:53.895165+00:00,Active


## 6. Load external system's view of EOD positions
We can call the LUSID `SetHoldings` API again, this time with the built-in `LDN_Close` cut label to set the datetime of each adjustment transaction to precisely 4:30pm UTC on 7 March 2022.

In [11]:
# Load end-of-day positions into LUSID effective 4:30pm UTC
load_positions(EOD_positions_df, "LDN_Close")

,instrument_uid,Name(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Figi,transaction.instrument_scope,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/ResultantHolding.key,transaction.properties.Transaction/default/ResultantHolding.value.metric_value.value,transaction.entry_date_time,transaction.transaction_status,transaction.cancel_date_time
0,LUID_00003D94,BP,P,"14,900.00","4,900.00","29,800.00",GBP,"29,800.00",NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,LUID_00003D95,Unilever,P,"1,010.00","4,010.00","3,030.00",GBP,"3,030.00",NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,CCY_GBP,GBP,B,"38,000.00","38,000.00","38,000.00",GBP,"38,000.00",NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,CCY_GBP,GBP,C,"-20,000.00",0.00,"-20,000.00",GBP,"-20,000.00",MD32001,Buy,BBG000C05BD1,FBNUniversityModule-3-2,LUID_00003D94,2022-03-07 12:00:00+00:00,2022-03-09 12:00:00+00:00,"10,000.00",2.00,Price,"20,000.00",GBP,1.00,GBP,Transaction/default/ResultantHolding,"15,000.00",2022-09-23 09:42:53.895165+00:00,Active,0001-01-01 00:00:00+00:00
4,CCY_GBP,GBP,C,"9,000.00",0.00,"9,000.00",GBP,"9,000.00",MD32002,Sell,BBG000C0M8X7,FBNUniversityModule-3-2,LUID_00003D95,2022-03-07 12:10:00+00:00,2022-03-09 12:10:00+00:00,"3,000.00",3.00,Price,"9,000.00",GBP,1.00,GBP,Transaction/default/ResultantHolding,"1,000.00",2022-09-23 09:42:53.895165+00:00,Active,0001-01-01 00:00:00+00:00


## 7. Reconcile LUSID's view vs. external system's view

We can call the LUSID `ReconcileHoldings` API and:

* On the left-hand side pass in the portfolio at 4:29pm on 7 March 2022, which is LUSID's calculation of holdings.
* On the right-hand side pass in the same portfolio at 4:30pm (`LDN_Close`), which is the external system's view of positions.

In [12]:
# Obtain the Reconciliations API
reconciliations_api=api_factory.build(la.ReconciliationsApi)

# Create reconcilation request
reconcile_holdings_request = lm.PortfoliosReconciliationRequest(
    # Pass in the portfolio at 4:29pm on the left
    left = lm.PortfolioReconciliationRequest(
        portfolio_id = lm.ResourceId(
            scope = module_scope,
            code = module_code
        ),
        # Note 'T' separator between date and explicit time
        effective_at = "2022-03-07T16:29:00Z"
    ),
    # Pass in the portfolio at LDN_Close, which is 4:30pm, on the right
    right = lm.PortfolioReconciliationRequest(
        portfolio_id = lm.ResourceId(
            scope = module_scope,
            code = module_code
        ),
        # Note 'N' separator between date and cut label time
        effective_at = "2022-03-07NLDN_Close"
    ),
    # Decorate on instrument name property to make results more intuitive
    instrument_property_keys = ["Instrument/default/Name"]
)

# Reconcile holdings
get_reconciliation_response = reconciliations_api.reconcile_holdings(
    portfolios_reconciliation_request = reconcile_holdings_request
)

# Transform API response to a Pandas dataframe and show it
get_reconciliation_response_df=lusid_response_to_data_frame(get_reconciliation_response, rename_properties=True)
# Drop some noisy columns
get_reconciliation_response_df.drop(columns=[
    "sub_holding_keys", "instrument_properties.0.effective_from", "instrument_properties.0.key"], inplace=True)
display(get_reconciliation_response_df)

,instrument_scope,instrument_uid,left_units,right_units,difference_units,left_cost.amount,left_cost.currency,right_cost.amount,right_cost.currency,difference_cost.amount,difference_cost.currency,instrument_properties.0.value.label_value
0,FBNUniversityModule-3-2,LUID_00003D94,"15,000.00","14,900.00",-100.00,"30,000.00",GBP,"29,800.00",GBP,-200.00,GBP,BP
1,FBNUniversityModule-3-2,LUID_00003D95,"1,000.00","1,010.00",10.00,"3,000.00",GBP,"3,030.00",GBP,30.00,GBP,Unilever
2,default,CCY_GBP,"39,000.00","27,000.00","-12,000.00","39,000.00",GBP,"27,000.00",GBP,"-12,000.00",GBP,GBP
